# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** The students intervention system is a typical classification problem. The goal of the system is, for each student, decide if he/she needs an early intervention or not, in other words, for each of the many students we have to classify he/she as 'needs intervention' or 'does not need intervention'. In this situation, when we need to separate data (students) in different classes, we have a Classification type of problem, as opposed to when we want to predict a continuous output such as the amount of rain that is going to fall in a given day, which is a Regression type of problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]

# TODO: Calculate passing students
n_passed = student_data[student_data.passed == "yes"].shape[0]

# TODO: Calculate failing students
n_failed = student_data[student_data.passed == "no"].shape[0]

# TODO: Calculate graduation rate
grad_rate = n_passed / float(n_students) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn import cross_validation

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, test_size=num_test, random_state=0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: ** The first algorithm I chose was the **Support Vector Machine** (SVM). One of the main uses for SVMs is finding a separate line between the classes in the data, and what the SVM will always try to do is to maximize the Margin which is the distance from the decision boundary to the points of the classes, even if some points get misclassified (generally those misclassified data points are outliers, which makes SVMs good models for outliers detection). Given my insites about the data, I have a feeling that SVMs might be a good fit given that SVMs are also capable of finding separate lines even in high dimensional spaces, which is the case of the stutents dataset. Also, according to sklearn documentation, SVMs actually do preaty well in high dimentional datasets, and another great advantage of this model is the fact that it is memory efficient. On the other hand, according to the sklearn documentation, one of the disadvantages of SVMs is that it might have poor performance when the number of features in the dataset is much greater than the number of samples, fortunately, this is not the case with the students dataset. From my perspective, it might be reasonable to think that there might exist a decision boundary between students who passed and the ones who failed. And because that line does not need to happen in a two dimension environment, SVM might be a good candidate to find it. 

The second model chosen is the **Naive Bayes** classifier. Naive bayer has a long and classical history of processing text data. As shown in the class, Naive bayes can easily been used to identify who wrote an email based on the probability of the words each possible writter most uses, or try to identify if an email is spam or not based on the probability of the words been used in spam emails previously. One of the advantages of this model, as we can see in the table bellow, is the efficiency in training the model. As we know, the number of paramenters used to train a Naive Bayes model is fairly small - the number of attributes *n* times the number of possible classes (in this case 2 - 'yes' or 'no'). Another great feature of Naive Bayes is that it is not sensitive of irrelevant features. For example, in the student dataset we have the feature 'sex' which in my opinion does not play an important role in deciding wheather or not a student will pass his/her final exam, so in this line, the probability of passing the final exam given that the student is a boy or a girl would be almost the same. However, the reason why this model is called naive is that it assumes that all of the attributes are conditionally independent given we know the label. In other words, Naive Bayes does not model the interrelationships between the attributes because it assumes one does not depend on others which might desqualify this approach for a given dataset. From my point of view, Naive Bayes can be a very good fit to the student dataset because even though it makes some assumptions regarding the independency of the attributes, in practice this does not, significantly, degrades the models accuracy. Also, it might be reasonable thinking that the combination of the individial features's probabilities, given the label, might find a good decision boundary between the classes.

The third algorithm chosen for the task at hand is the **K Nearest Neighbors algorithm** (K-NN). In the two algorithms described above, SVM and Naive Bayes, the training data is fit into the classifier which tries to model or learn a function that best categorizes the training data. After that, when attempting to predict a new data instance, this new data instance is fed into the function which outputs the learned class based on the data it has seen, so they are called eager learners because after the model is trained and the best function that represents the data is learned, the data it self is not anymore necessary. Different from that, the K-NN algorithm is a lasy learner because it does not take the time of first looking at the data, learning a function, and then predicting, rather, it uses the data it self as the function in which predictions are made as they are requested. Of course one of the main advantages of this model is that the time taken to learn is not needed since the data is used as the function. Also, the absense of a learning phase also contributes to the possibility of increasing the training data at any time with no extra cost. On the other hand, because at any given time a new data point needs to be evaluated the algorithm goes over the training set to classify it, the amount of time predicting is much greater. Indeed, for a value of k greater than 0, the time taken to predict a value is O(lg(n) + k), where lg is log base 2, whereas in most of the eager learner algorithms this time is constant - 0(1). Also, K-NN suffers a lot from the curse of dimensionaly because depending of the similarity function chosen, K-NN treats the parameters equally, that is, when predicting a new data point the features of the nearest data points matter, but they matter equally. Finally, given my thoughts about the data, I believe K-NN may be a good fit because in essence it assumes that the points near each other, based on some similarity measure, behavies similarly, in my oppinion, it makes sense to think that maybe, amongst the students that succeeded the final exam, there are commom features that might represent them as a group, the same for the ones that failed.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn.svm import SVC

# from sklearn import model_B
from sklearn.naive_bayes import GaussianNB

# from skearln import model_C
from sklearn.neighbors import KNeighborsClassifier

seed = 42

# TODO: Initialize the three models
clf_A = SVC(random_state=seed)
clf_B = GaussianNB()
clf_C = KNeighborsClassifier()


# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a SVC using a training set size of 100. . .
Trained model in 0.0027 seconds
Made predictions in 0.0018 seconds.
F1 score for training set: 0.8591.
Made predictions in 0.0019 seconds.
F1 score for test set: 0.7838.
Training a SVC using a training set size of 200. . .
Trained model in 0.0043 seconds
Made predictions in 0.0040 seconds.
F1 score for training set: 0.8693.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7755.
Training a SVC using a training set size of 300. . .
Trained model in 0.0116 seconds
Made predictions in 0.0084 seconds.
F1 score for training set: 0.8692.
Made predictions in 0.0047 seconds.
F1 score for test set: 0.7586.
Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0013 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.8550.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.7481.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0011 seconds
Made pred

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Support Vector Machine**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0029 s       |     0.0010 s           |       0.8591     |      0.7838     |
| 200               |          0.0039 s       |     0.0013 s           |       0.8693     |      0.7755     |
| 300               |          0.0058 s       |     0.0024 s           |       0.8692     |      0.7586     |

** Classifer 2 - Naive Bayes**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0006 s        |        0.0004 s        |      0.8550      |      0.7481     |
| 200               |         0.0004 s        |        0.0004 s        |      0.8321      |      0.7132     |
| 300               |         0.0009 s        |        0.0006 s        |      0.8088      |      0.7500     |

** Classifer 3 - K Nearest Neighbors**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0020 s        |       0.0022 s         |     0.7972       |      0.7068     |
| 200               |         0.0043 s        |       0.0033 s         |     0.8571       |      0.7121     |
| 300               |         0.0087 s        |       0.0028 s         |     0.8722       |      0.7482     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** Based on the experiements made and on the coditions stated in the question, I decided that the SVM classifier model is the most apropriante for this task, between the three discussed above. Based on the results reported on the table above, the SVM classifier proved to be the most consistent model when it comes to efficiency and effectiveness. We can see that because it is an eager lerner model, it takes (overraw) less time to predict a new point when compared with the lasy learner K-NN, but it takes longer than the Gausian Naive Bayes classifier, which is known to be a very light weight model. When it comes to F1 scores, the SVM accomplishes the best numbers (overraw) when training with all three possible training sizes on the testing and trainig sets. Although SVMs takes longer to predict new data instances when compared to the Gausian Naive Bayes, given our limited dataset size, I do not consider this difference in performane sufficient enough for using NB over SVM. Thus, as we can see in the table above, and in the outpus, the training and predicting costs are not a deal breaker between these two models. Also it is important to note that, based on the available data, although SVMs can take on the problem of multi-class classification, it is important to emphasize that the natural task for SVMs is binary classification, which suits our student dataset perfectly.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: ** The Support Vector Machine model chosen for the task of predicting if students will or will not pass their final exam works by trying to identify a decision boundary that classifies the training points by their class labels. In the students dataset, to be more specific, the SVM will try to draw a separating line that divides the students who passed on their final exams from the ones who do not. However, because there might be more than one of these lines, the SVM classifier will try to identify the best one using the criteria of maximizing the Margin. Once the many possible separating lines are identified, the Margin can be thought as the distance between these lines and the two closest points that lie on opposite margins, and therefore, ideally the SVM will try to pick the one line that maximizes the size of the Margin. However, this approach describes how to compute SVMs in datasets that are divisible by a line, also called linearly separable datasets,but we know that most of the datasets are not easily separable by a straight line, which is the case of the students dataset. For datasets that are not divived in a linearly fashion, SMVs usually presents two approaches, the first one attempts to divide the dataset in a liniearly fashion even at the cost of some points getting misclassified, this is called soft-margin approach and it might be a good strategy if the data is mostly linearly separable. The second approach aims to utilie special functions called kernels, which will make a harder effort of modeling nomlinear dividing curves between classes. In the sklearn SVC class, we can tune the models funcition by changing the kernel parameter. In conclusion, once the SVM learned the best function representation based on the training dataset, we can fed this function with an unseen data point (student's features such as age, address, etc) and the function will classify it as passed or not based on what it has learned.    

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [57]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
C_range = np.logspace(-2, 1, 32)
gamma_range = np.logspace(-9, 3, 46)
print C_range
# Set the parameters by cross-validation
parameters = {'kernel': ['rbf'], 'gamma': [0.06], 'C': [1.06]}

# TODO: Initialize the classifier
clf = SVC(random_state=seed, decision_function_shape='ovr' )

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(score_func=f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer, n_jobs=4)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print("The best parameters are %s with a score of %0.2f" % (grid_obj.best_params_, grid_obj.best_score_))
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

[  0.01         0.01249609   0.01561523   0.01951293   0.02438354
   0.0304699    0.03807546   0.04757944   0.05945571   0.0742964
   0.09284145   0.11601553   0.14497407   0.18116092   0.22638034
   0.28288694   0.35349811   0.44173447   0.55199543   0.68977854
   0.86195357   1.07710506   1.34596032   1.68192432   2.10174801
   2.62636353   3.28192787   4.10112707   5.12480588   6.40400427
   8.00250228  10.        ]
The best parameters are {'kernel': 'rbf', 'C': 1.06, 'gamma': 0.06} with a score of 0.83
Made predictions in 0.0059 seconds.
Tuned model has a training F1 score of 0.9406.
Made predictions in 0.0019 seconds.
Tuned model has a testing F1 score of 0.8000.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The final F1 score for training and testing datasets are 0.9406 and 0.8 respectively. Comparing the F1 scores reported on the untuned model and the ones on the tuned model, we can see an improvement in both cases. On the training dataset, there was an improvement of nearly 8% while in the testing set the improvemnt was roughly 5%. Although some improvement was accomplished in the tuned model, I believe that some important considerations and decisions blocked the model from getting a higher improvement. To begin with, I beleive that the creation of most of the dummy variables contributed for the curse of dimensionaly which says that every time a new feature is added, the amount of data samples needed for a model's learning increases exponentially. In the *processing feature columns* section the number of features changes from 31 (originally) to 48 after dummy variables creation. I believe that this strategy (dummy variables) could be avoided using the same strategy done with the 'sex' feature where numbers, such as 0s and 1s substituted the categorical variables. Also, once we got this fairly big number of features with a relatively low number of samples (300 for training), it made the work of fitting a function on this dataset much harder. To avoid that, some of the features could've been left out of the processing since, in my opinion, some of them are not decisive to predict the desired outcome.     

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.